In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
columns = ['Area', 'Perimeter', 'Compactness', 'KernelLength', 'KernelWidth',
           'AsymmetryCoeff', 'KernelGrooveLength', 'Class']

In [3]:
df = pd.read_csv('seeds_dataset.txt', sep='\s+', names=columns)


In [4]:
df

,Area,Perimeter,Compactness,KernelLength,KernelWidth,AsymmetryCoeff,KernelGrooveLength,Class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [5]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values - 1

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
X = np.expand_dims(X, axis=-1)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv1D(16, kernel_size=3, activation='relu', input_shape=(7, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

In [16]:
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1, verbose=1, callbacks=[early_stopping])

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3496 - loss: 1.0730 - val_accuracy: 0.9412 - val_loss: 0.8408
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7299 - loss: 0.9054 - val_accuracy: 0.8824 - val_loss: 0.7076
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7857 - loss: 0.8131 - val_accuracy: 0.8824 - val_loss: 0.6021
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8373 - loss: 0.7090 - val_accuracy: 0.8824 - val_loss: 0.5008
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8316 - loss: 0.6076 - val_accuracy: 0.8824 - val_loss: 0.4265
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8627 - loss: 0.5345 - val_accuracy: 0.9412 - val_loss: 0.3486
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8666 - loss: 0.4728 - val_accuracy: 1.0000 - val_loss: 0.2977
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8411 - loss: 0.4560 - val_accuracy: 1.0000 - val_loss:

In [19]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Sequential 모델 정확도: {acc*100:.2f}%")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8631 - loss: 0.2781
Sequential 모델 정확도: 85.71%
